<a href="https://colab.research.google.com/github/itsmeeeeeee/BERT_Probing/blob/main/embedding_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.6 MB/s eta 0:00:00


In [2]:
import json

import pandas as pd
import numpy as np
from ast import literal_eval
from transformers import BertTokenizer, BertModel, AutoTokenizer
import torch
from pathlib import Path
from typing import Dict
import os
import glob
import time
import traceback

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model_ckpt = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_ckpt)
# tokenizer.model_max_length = 518
model = BertModel.from_pretrained(model_ckpt)


In [5]:

# Returns a list of lists of indices for each subtoken from the tokenizer, if more than one subtoken appears

def get_indexof_subtokens(tokenizer_output):
    result = []
    temp_liste = [tokenizer_output[0]]  # Start with the first word in the temporary list

    for i in range(1, len(tokenizer_output)):
        # If the current word has the same property as the previous word
        if tokenizer_output[i][:2] == "##" and tokenizer_output[i - 1][:2] == "##":
            temp_liste.append(i)



        else:
            if len(temp_liste) > 1:  # If the temporary list contains more than one word, add it to the result.
                result.append(temp_liste)
            temp_liste = [i]  # Start a new temporary list with the current word.

    # Add the last temporary list to the result if it contains more than one word
    if len(temp_liste) > 1:
        result.append(temp_liste)

    return result

ll = ['[CLS]', 'Celebration', 'Shooting', 'Turn', '##s', 'Wedding', 'Into', 'a', 'Fun', '##eral', 'in', 'Southern',
      'Gaza', 'Strip', 'As', '##ad', '1', '/', '20', '/', '2007', 'Gaza', '-', 'UP', '##I', 'The', 'cheer', '##s',
      'and', 'hail', '##s', 'of', 'happiness', 'at', 'a', 'wedding', 'in', 'Khan', 'You', '##nes', 'in', 'the',
      'southern', 'Gaza', 'Strip', 'turned', 'into', 'screams', 'and', 'moan', '##s', 'of', 'pain', 'after', 'one',
      'of', 'the', 'c', '##ele', '##bra', '##tors', 'lost', 'control', 'of', 'his', 'weapon', ',', 'from', 'which', 'a',
      'number', 'of', 'bullets', 'were', 'released', 'that', 'killed', 'the', 'groom', "'", 's', 'brother', 'and',
      'hit', 'three', 'other', 'relatives', 'of', 'his', ',', 'turning', 'the', 'wedding', 'into', 'a', 'funeral', 'in',
      'moments', 'c', '##ele', '##bra', '##tors', '[SEP]']

get_indexof_subtokens(ll)

[[58, 59, 60], [99, 100, 101]]

In [6]:


"""Returns three lists: one list with main words (only those tokenized into a single subword),
   one with subword related to the main word, and one list with tokenizer indices of the main words.
    All three lists are of the same length"""


def get_main_token_subtoken_embed_index(tokens):
    main_token = []
    subtoken = []
    embed_index_main_token = []

    for i in range(len(tokens) - 1):
        s = ""
        if "##" in tokens[i] and "##" not in tokens[i - 1] and "##" not in tokens[i + 1]:
            s += tokens[i - 1]
            subtoken.append(tokens[i])
            s += tokens[i]
            embed_index_main_token.append(i - 1)


        main_token.append(s.replace("##", ""))
    main_token = [x for x in main_token if x != ""]
    return main_token, subtoken, embed_index_main_token


ll = ['[CLS]', 'Celebration', 'Shooting', 'Turn', '##s', 'Wedding', 'Into', 'a', 'Fun', '##eral', 'in', 'Southern',
      'Gaza', 'Strip', 'As', '##ad', '1', '/', '20', '/', '2007', 'Gaza', '-', 'UP', '##I', 'The', 'cheer', '##s',
      'and', 'hail', '##s', 'of', 'happiness', 'at', 'a', 'wedding', 'in', 'Khan', 'You', '##nes', 'in', 'the',
      'southern', 'Gaza', 'Strip', 'turned', 'into', 'screams', 'and', 'moan', '##s', 'of', 'pain', 'after', 'one',
      'of', 'the', 'c', '##ele', '##bra', '##tors', 'lost', 'control', 'of', 'his', 'weapon', ',', 'from', 'which', 'a',
      'number', 'of', 'bullets', 'were', 'released', 'that', 'killed', 'the', 'groom', "'", 's', 'brother', 'and',
      'hit', 'three', 'other', 'relatives', 'of', 'his', ',', 'turning', 'the', 'wedding', 'into', 'a', 'funeral', 'in',
      'moments', 'c', '##ele', '##bra', '##tors', '[SEP]']
main_token, subtoken, embed_index = get_main_token_subtoken_embed_index(ll)
print(main_token)
print(subtoken)
print(embed_index)

['Turns', 'Funeral', 'Asad', 'UPI', 'cheers', 'hails', 'Younes', 'moans']
['##s', '##eral', '##ad', '##I', '##s', '##s', '##nes', '##s']
[3, 8, 14, 23, 26, 29, 38, 49]


In [7]:

"""Returns three lists: one list with main words (only those tokenized into multi subwords),
   one with list of subwords related to the main words, and one list with tokenizer indices of the main words.
    All three lists are of the same length"""

def get_main_tokens_subtokens_embed_index(tokenizer_output):
    index_subtokens = get_indexof_subtokens(tokenizer_output)
    main_words = []
    subwords = []
    embed_index = []
    s = ""
    for inds in index_subtokens:
        #print(".......",res[0]-1)
        s += tokenizer_output[inds[0] - 1]
        l = []
        for ind in inds:
            l.append(tokenizer_output[ind])

            s += tokenizer_output[ind]
        subwords.append(l)
        main_words.append(s.replace("##", ""))
        s = ""
        embed_index.append(inds[0] - 1)

    return main_words, subwords, embed_index

ll = ['[CLS]', 'Celebration', 'Shooting', 'Turn', '##s', 'Wedding', 'Into', 'a', 'Fun', '##eral', 'in', 'Southern',
      'Gaza', 'Strip', 'As', '##ad', '1', '/', '20', '/', '2007', 'Gaza', '-', 'UP', '##I', 'The', 'cheer', '##s',
      'and', 'hail', '##s', 'of', 'happiness', 'at', 'a', 'wedding', 'in', 'Khan', 'You', '##nes', 'in', 'the',
      'southern', 'Gaza', 'Strip', 'turned', 'into', 'screams', 'and', 'moan', '##s', 'of', 'pain', 'after', 'one',
      'of', 'the', 'c', '##ele', '##bra', '##tors', 'lost', 'control', 'of', 'his', 'weapon', ',', 'from', 'which', 'a',
      'number', 'of', 'bullets', 'were', 'released', 'that', 'killed', 'the', 'groom', "'", 's', 'brother', 'and',
      'hit', 'three', 'other', 'relatives', 'of', 'his', ',', 'turning', 'the', 'wedding', 'into', 'a', 'funeral', 'in',
      'moments', 'c', '##ele', '##bra', '##tors', '[SEP]']
main_words, subwords, embed_index = get_main_tokens_subtokens_embed_index(ll)
print(main_words)
print(subwords)  #teilwort,embed_index)
print(embed_index)

['celebrators', 'celebrators']
[['##ele', '##bra', '##tors'], ['##ele', '##bra', '##tors']]
[57, 98]


In [8]:
#The data is read from Google Drive
file_path = "/content/drive/My Drive/embedding_data/final_endlabels_with_sentences.csv"

final_df = pd.read_csv(file_path)


<ipython-input-8-eccc40b3a099>:9: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv(file_path)


In [9]:
# Initializes the two new columns
final_df["Sentence_row"] = np.NaN
final_df["Word_count"] = np.NaN

In [10]:
#Sentence_row" for sentences in string format and "Word_count" for the number of words in each sentence."""


start_time = time.time()

batch_size = 1000000
num_batches = len(final_df) // batch_size

# Process the data in batches
for batch_num in range(num_batches + 1):
    # Create a subset for the current batch
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_df = final_df.iloc[start_idx:end_idx]

    for index , row in batch_df.iterrows():
        sen = row['Sentence'].strip(")(").split(", ")
        sen_row = ""
        for word in sen:
            wo = word[1:-1]
            sen_row += wo + " "
        # print(sen_row)
        batch_df.at[index,"Sentence_row"] = sen_row
        batch_df.at[index,"Word_count"] = len(sen)
    batch_df.to_csv(f"/content/drive/My Drive/embedding_data/updated_data_batch_with_word_count_{batch_num}.csv")

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60

print(f"Der Code hat {elapsed_time_minutes:.2f} Minuten gedauert.")


Der Code hat 11.73 Minuten gedauert.


In [12]:
# Concatenate all 5 batches into one dataframe

start_time = time.time()
list_of_files=[]
for batch_num in range(6):
    list_of_files.append(f"/content/drive/My Drive/embedding_data/updated_data_batch_with_word_count_{batch_num}.csv")

concat_with_sentences_word_count = pd.DataFrame()
for df_file in list_of_files:
    update_bach=pd.read_csv(df_file, low_memory=False)
    concat_with_sentences_word_count = pd.concat([concat_with_sentences_word_count, update_bach], ignore_index=True)


# Save DataFrame as a CSV file
concat_with_sentences_word_count.to_csv('/content/drive/My Drive/embedding_data/final_endlabels_with_sentences_word_count.csv', index=False)

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60

print(f"Der Code hat {elapsed_time_minutes:.2f} Minuten gedauert.")

Der Code hat 1.94 Minuten gedauert.


In [13]:
final_df = pd.read_csv("/content/drive/My Drive/embedding_data/final_endlabels_with_sentences_word_count.csv",low_memory=False)

In [14]:
final_df

,Unnamed: 0.1,Unnamed: 0,Sentence,Sentence_no,Word,POS,coref_id,antecedent,hypernyms,labels,end_labels,Sentence_row,Word_count
0,0,0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Celebration,PROPN,NaN,NaN,"['psychological_feature', 'social_event', 'abs...",inanimate,NaN,Celebration Shooting Turns Wedding Into a Fune...,82.0
1,1,1,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Shooting,PROPN,NaN,NaN,"['psychological_feature', 'act', 'abstraction'...",inanimate,NaN,Celebration Shooting Turns Wedding Into a Fune...,82.0
2,2,2,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Turns,VERB,NaN,NaN,NaN,other_pos,NaN,Celebration Shooting Turns Wedding Into a Fune...,82.0
3,3,3,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Wedding,NOUN,1__2,NaN,"['psychological_feature', 'social_event', 'abs...",inanimate,NaN,Celebration Shooting Turns Wedding Into a Fune...,82.0
4,4,4,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Into,ADP,NaN,NaN,NaN,other_pos,NaN,Celebration Shooting Turns Wedding Into a Fune...,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5910736,5910736,5910736,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,drop,VERB,NaN,NaN,NaN,other_pos,NaN,Barak is facing mounting pressure ICH from wit...,44.0
5910737,5910737,5910737,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,out,ADP,NaN,NaN,NaN,other_pos,NaN,Barak is facing mounting pressure ICH from wit...,44.0
5910738,5910738,5910738,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,of,ADP,NaN,NaN,NaN,other_pos,NaN,Barak is facing mounting pressure ICH from wit...,44.0
5910739,5910739,5910739,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,the,DET,NaN,NaN,NaN,other_pos,NaN,Barak is facing mounting pressure ICH from wit...,44.0


In [15]:
"""
This sentence appears twice consecutively in the dataset and has the same sentence number.
This causes issues with indexing the words within a sentence. Refer to the function "find_index_2".Therefore, it is deleted."""


final_df[final_df["Word"]=="worst"][100:150]
final_df[(final_df.index == 5222032)]["Sentence"].tolist()[0]
f_d=final_df[(final_df["Sentence_no"] != 264185)]

In [16]:
# Filters out the unnecessary columns for further processing
select_columns = f_d[["Sentence_row","Word_count","Sentence", "Sentence_no", "Word", "POS", "end_labels"]]
select_columns


,Sentence_row,Word_count,Sentence,Sentence_no,Word,POS,end_labels
0,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Celebration,PROPN,NaN
1,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Shooting,PROPN,NaN
2,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Turns,VERB,NaN
3,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Wedding,NOUN,NaN
4,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Into,ADP,NaN
...,...,...,...,...,...,...,...
5910736,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,drop,VERB,NaN
5910737,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,out,ADP,NaN
5910738,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,of,ADP,NaN
5910739,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,the,DET,NaN


In [17]:
select_columns.shape

(5910725, 7)

In [18]:


""" 1. Sentences longer than 5 words are selected because the same sentences frequently appear in the dataset with same sentence numbers.
This complicates the mapping of the embedding vector to the word in the dataset."""

""" 2. Sentences shorter than 500 words are selected to ensure that the allowed length of the BERT tokenizer is not exceeded."""

filtered_df = select_columns[(select_columns['Word_count'] < 500) & (select_columns['Word_count'] > 5)]


In [19]:
len(filtered_df.groupby(["Sentence_row"])["Sentence_row"])


35020

In [20]:
"""The function returns the main information for the correct assignment of the embedding vector
  to each word in the respective sentence. It returns a list of main tokens, a list of subtokens,
  a list of indices for the assignment of the embedding vector, a list for all tokens, and a list
  with all contextualized vectors for a sentence."""

def extract_main_info_embedding_vec(texts_ohne_dublikate):
    all_main_tokens = []
    all_subtokens = []
    all_embeded_index = []
    all_tokens = []
    all_contextualized_vectors = []
    for ind, el in enumerate(texts_ohne_dublikate):

        tokenized_input = tokenizer(el)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
        #print(el)
        #print(tokens)

        main_tokens, subtokens, embed_index_main_tokens= get_main_tokens_subtokens_embed_index(tokens)
        main_token, subtoken, embed_index_main_token = get_main_token_subtoken_embed_index(tokens)
        main_tokens += main_token
        subtokens += subtoken
        embed_index_main_tokens += embed_index_main_token
        all_tokens.append(tokens)
        all_main_tokens.append(main_tokens)
        all_subtokens.append(subtokens)
        all_embeded_index.append(embed_index_main_tokens)

        with torch.no_grad():
            outputs = model(torch.tensor(tokenized_input["input_ids"]).unsqueeze(0))
            contextualized_vectors = outputs.last_hidden_state
            for el in contextualized_vectors:
                all_contextualized_vectors.append(el)



        #print(f"""##########################################################################
                ############################# new element {ind} ##############################
        ########################################################################""")
    return all_main_tokens,all_subtokens,all_embeded_index,all_tokens,all_contextualized_vectors

#all_main_tokens,all_subtokens, all_embeded_index, all_tokens, all_contextualized_vectors=extract_main_info_embedding_vec(texts_ohne_dublikate)


In [21]:
# Initialize three new columns
filtered_df['subword'] = None
filtered_df['wort_index'] = None
filtered_df['contextualized_vectors'] = None

<ipython-input-21-7a7f663c5fe2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['subword'] = None
<ipython-input-21-7a7f663c5fe2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['wort_index'] = None
<ipython-input-21-7a7f663c5fe2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [22]:
"""
    We iterate through the dataframe using a for loop. For all rows that have the same sentence,
    we define a dictionary called 'word_already_seen', so that words that appear more than once
    in a sentence don't receive the same index,
    but rather a unique index.

"""

def find_index_2(curr_word: str, sub_tokens: list, already_seen: Dict):

    last_index = already_seen.get(curr_word, 0)
    curr_index = sub_tokens.index(curr_word, last_index + 1)
    already_seen[curr_word] = curr_index
    return curr_index

In [23]:
"""This function is the main function for assigning 'subwords', word_index,
and contextualized vectors to each word in the dataframe. Data is processed in batches"""


start_time = time.time()

batch_size = 100000
num_batches = len(filtered_df) // batch_size

# Process the data in batches
for batch_num in range(num_batches + 1):
    # Create a subset for the current batch
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_df = filtered_df.iloc[start_idx:end_idx]
    # Raw sentences are saved without duplicates
    texts = batch_df["Sentence_row"].tolist()
    texts_ohne_dublikate = pd.Series(texts).dropna().drop_duplicates().tolist()
    all_main_tokens,all_subtokens, all_embeded_index, all_tokens, all_contextualized_vectors=extract_main_info_embedding_vec(texts_ohne_dublikate)

    old_sentence_list = ""
    counter = -1
    main_tokens_fuer_current_satz = []
    subtokens_fuer_current_satz = []
    embed_index_fuer_current_satz = []
    tokens_fuer_current_satz = []
    previous_tokens = []
    previous_sentence_no = None
    word_already_seen = Dict

    tensor_output = []
    for index, row in batch_df.iterrows():
        sentence_list = row["Sentence_row"]

        """"Search for the index in 'texts_without_duplicates'. This is important because it corresponds to the
        index for contextualized vectors, subwords, main_words, and EmbeddingIndex """

        #all_main_tokens,all_subtokens, all_embeded_index, all_tokens, all_contextualized_vectors=extract_main_info_embedding_vec(texts_ohne_dublikate)
        index_of_Sentence = texts_ohne_dublikate.index(sentence_list)
        tokens_fuer_current_satz = all_tokens[index_of_Sentence]
        main_tokens_fuer_current_satz = all_main_tokens[index_of_Sentence]
        subtokens_fuer_current_satz = all_subtokens[index_of_Sentence]
        embed_index_fuer_current_satz = all_embeded_index[index_of_Sentence]


        sentence_no_fuer_current_satz = row['Sentence_no']
        contextualized_vectors_fuer_current_satz = all_contextualized_vectors[index_of_Sentence]

        """When we come across a new sentence or the same sentence with a different sentence number,
        the 'word_already_seen' is emptied """

        if previous_tokens != tokens_fuer_current_satz or previous_sentence_no != sentence_no_fuer_current_satz:
            #print(f"changing from S_no:{previous_sentence_no} to-> S_no:{sentence_no_fuer_current_satz}")
            word_already_seen = dict()
        current_word = row['Word']
        index_of_word = -1
        if current_word in main_tokens_fuer_current_satz:
            index_of_hauptwort =  main_tokens_fuer_current_satz.index(current_word)
            batch_df.at[index, 'subword'] = subtokens_fuer_current_satz[index_of_hauptwort]
            index_of_word = embed_index_fuer_current_satz[index_of_hauptwort]
            batch_df.at[index, "wort_index"] = index_of_word

        elif current_word in tokens_fuer_current_satz:
            try:
                index_of_word = find_index_2(current_word, tokens_fuer_current_satz, word_already_seen)
            except Exception as e:
                print(f"""*****************************************************************************
            ********************************** index {index} *******************************************
            *************************************************************************************""")
                #print(index)
                #print("current_word", current_word)
                #print("tokens_fuer_current_satz:")
                #print(tokens_fuer_current_satz)
                #print("word already seen")
                #print(word_already_seen)
                traceback.print_exc()
                break
            batch_df.at[index, "wort_index"] = index_of_word

        if index_of_word != -1:

            # If 'current_word' appears in main_word or in 'tokens', then the index must be a value that is >= 0


            e = contextualized_vectors_fuer_current_satz[index_of_word].tolist()
            tensor_output.append(e)

        else:

            # For the word (which was previously tokenized in our dataframe using spaCy), the tokenizer couldn't generate a value
            tensor_output.append(None)


        previous_tokens = tokens_fuer_current_satz
        previous_sentence_no = sentence_no_fuer_current_satz

    print("hallo there")

    batch_df['contextualized_vectors'] = tensor_output


    # Save the updated dataframe for the current batch
    batch_df.to_csv(f"/content/drive/My Drive/embedding_data/updated_data_batch_embedding{batch_num}.csv")


    print(f"Fertig_{batch_num}")

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60

print(f"Der Code hat {elapsed_time_minutes:.2f} Minuten gedauert.")


hallo there


<ipython-input-23-154eacba1e4f>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['contextualized_vectors'] = tensor_output


Fertig_0
hallo there
Fertig_1
hallo there
Fertig_2
hallo there
Fertig_3
hallo there
Fertig_4
hallo there
Fertig_5
hallo there
Fertig_6
hallo there
Fertig_7
hallo there
Fertig_8
hallo there
Fertig_9
hallo there
Fertig_10
hallo there
Fertig_11
hallo there
Fertig_12
hallo there
Fertig_13
hallo there
Fertig_14
hallo there
Fertig_15
hallo there
Fertig_16
hallo there
Fertig_17
hallo there
Fertig_18
hallo there
Fertig_19
hallo there
Fertig_20
hallo there
Fertig_21
hallo there
Fertig_22
hallo there
Fertig_23
hallo there
Fertig_24
hallo there
Fertig_25
hallo there
Fertig_26
hallo there
Fertig_27
hallo there
Fertig_28
hallo there
Fertig_29
hallo there
Fertig_30
hallo there
Fertig_31
hallo there
Fertig_32
hallo there
Fertig_33
hallo there
Fertig_34
hallo there
Fertig_35
hallo there
Fertig_36
hallo there
Fertig_37
hallo there
Fertig_38
hallo there
Fertig_39
hallo there
Fertig_40
hallo there
Fertig_41
hallo there
Fertig_42
hallo there
Fertig_43
hallo there
Fertig_44
hallo there
Fertig_45
hallo ther

In [24]:
"""
  All data distributed in batches are read and cleaned up. The data is filtered by end labels
  (only 0 and 1 labels are taken). The NaN values from the "embedding vector" are removed.
  Due to the fact that many sentences in the dataset repeat themselves,
  similar embedding vectors also appear in the dataframe. However, they are removed here.

"""
start_time = time.time()
concat_filtered_embedding=[]
for bach_num in range(58):
    update_bach=pd.read_csv(f"/content/drive/My Drive/embedding_data/updated_data_batch_embedding{batch_num}.csv", low_memory=False)
    print(f"read_{bach_num}_baches")

    #filtered by end labels (only 0 and 1 labels are taken)
    update_bach_filtered_endlabels=update_bach[(update_bach["end_labels"] >= 0.0) & (update_bach["end_labels"] <= 1.0)]

    #the NaN values from the "contextualized_vectors" are removed
    update_bach_filtered_endlabels.dropna(subset=["contextualized_vectors"],inplace=True)

    # similar embedding vectors are remove
    end_embedding = update_bach_filtered_endlabels.drop_duplicates(subset="contextualized_vectors")
    concat_filtered_embedding.append(end_embedding)
    print(f"append_num_{bach_num}")
# concatenate all dataframes
filtered_embedding_data = pd.concat(concat_filtered_embedding,ignore_index=True)


# Save DataFrame as a CSV file
filtered_embedding_data.to_csv('/content/drive/My Drive/embedding_data/final_embedding_data.csv', index=False)

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60

print(f"Der Code hat {elapsed_time_minutes:.2f} Minuten gedauert.")

read_0_baches
append_num_0


<ipython-input-24-728af1b84ec5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update_bach_filtered_endlabels.dropna(subset=["contextualized_vectors"],inplace=True)


read_1_baches
append_num_1


<ipython-input-24-728af1b84ec5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update_bach_filtered_endlabels.dropna(subset=["contextualized_vectors"],inplace=True)


read_2_baches
append_num_2


...

read_56_baches
append_num_56


<ipython-input-24-728af1b84ec5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update_bach_filtered_endlabels.dropna(subset=["contextualized_vectors"],inplace=True)


read_57_baches
append_num_57


<ipython-input-24-728af1b84ec5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update_bach_filtered_endlabels.dropna(subset=["contextualized_vectors"],inplace=True)


Der Code hat 9.29 Minuten gedauert.


In [26]:
df_fin=pd.read_csv("/content/drive/My Drive/embedding_data/final_embedding_data.csv", low_memory=False)

In [28]:
df_fin.shape

(74414, 11)

In [34]:
df_fin

,Unnamed: 0,Sentence_row,Word_count,Sentence,Sentence_no,Word,POS,end_labels,subword,wort_index,contextualized_vectors
0,5842828,The Palestinian faction headed by Yasser Ara...,30.0,"('The', 'Palestinian', 'faction', 'headed', ' ...",303498,faction,NOUN,0.0,NaN,3.0,"[-0.2964571714401245, 0.02676987275481224, -0...."
1,5842838,The Palestinian faction headed by Yasser Ara...,30.0,"('The', 'Palestinian', 'faction', 'headed', ' ...",303498,it,PRON,0.0,NaN,15.0,"[-0.3028354048728943, -0.15795791149139404, -0..."
2,5842846,The Palestinian faction headed by Yasser Ara...,30.0,"('The', 'Palestinian', 'faction', 'headed', ' ...",303498,its,PRON,0.0,NaN,24.0,"[-0.48960378766059875, 0.4215056896209717, -0...."
3,5842982,Multinational companies investing in China had...,31.0,"('Multinational', 'companies', 'investing', 'i...",303506,companies,NOUN,0.0,NaN,3.0,"[0.2172500342130661, 0.2576604187488556, -0.01..."
4,5842991,Multinational companies investing in China had...,31.0,"('Multinational', 'companies', 'investing', 'i...",303506,they,PRON,0.0,NaN,12.0,"[-0.15129604935646057, -0.14134235680103302, -..."
...,...,...,...,...,...,...,...,...,...,...,...
74409,5910598,270 people died in the 1988 bombing over the S...,13.0,"('270', 'people', 'died', 'in', 'the', '1988',...",307826,bombing,NOUN,0.0,NaN,7.0,"[0.7685377597808838, 1.115256905555725, -0.038..."
74410,5910618,Libya initially refused to hand over the two...,35.0,"('Libya', 'initially', 'refused', ' ', 'to', '...",307827,them,PRON,0.0,NaN,13.0,"[0.11229103803634644, 0.16903077065944672, -0...."
74411,5910626,Libya initially refused to hand over the two...,35.0,"('Libya', 'initially', 'refused', ' ', 'to', '...",307827,they,PRON,0.0,NaN,21.0,"[0.4275742769241333, 0.07952270656824112, 0.25..."
74412,5910657,Opinion polls in Israel show hard - liner Arie...,27.0,"('Opinion', 'polls', 'in', 'Israel', 'show', '...",307829,liner,NOUN,0.0,NaN,10.0,"[0.5175453424453735, -0.12556852400302887, -0...."


In [ ]:
# following teil ist für debugging gedacht

#filtered_df[(filtered_df.index > 180000 ) & (filtered_df.index < 181180 )]
#filtered_df[(filtered_df.index == 156602)]
#filtered_df[(filtered_df['Sentence_no'] == 6184)]

